In [61]:
import unidecode
from nltk.translate.bleu_score import sentence_bleu
import random as rd

play_file_infixes = ['othello', 'antony-and-cleopatra', 'asyoulikeit', 
                     'errors', 'hamlet', 'henryv', 'juliuscaesar', 'lear', 'macbeth', 
                     'merchant', 'msnd', 'muchado', 'richardiii', 'romeojuliet', 
                     'shrew', 'tempest', 'twelfthnight']
play_file_infixes = ['othello', 'richardiii']

agg_original_tuples = []
agg_modern_tuples = []
path = "data/shakespeare/data/align/plays/merged/"

for infix in play_file_infixes:
    modern_tuples = unidecode.unidecode(open(path + infix + "_modern.snt.aligned").read()).split("\n")
    original_tuples = unidecode.unidecode(open(path + infix + "_original.snt.aligned").read()).split("\n")
    agg_original_tuples.extend(original_tuples)
    agg_modern_tuples.extend(modern_tuples)
# aligned_othello_style1 = "data/shakespeare/data/align/plays/merged/othello_modern.snt.aligned"
# aligned_othello_style2 = "data/shakespeare/data/align/plays/merged/othello_original.snt.aligned"

# aligned_ant_cleo_style1 = "data/shakespeare/data/align/plays/merged/antony-and-cleopatra_modern.snt.aligned"
# aligned_ant_cleo_style2 = "data/shakespeare/data/align/plays/merged/antony-and-cleopatra_original.snt.aligned"

# aligned_asyoulikeit_style1 = "data/shakespeare/data/align/plays/merged/asyoulikeit_modern.snt.aligned"
# aligned_asyoulikeit_style2 = "data/shakespeare/data/align/plays/merged/asyoulikeit_original.snt.aligned"

# aligned_errors_style1 = "data/shakespeare/data/align/plays/merged/errors_modern.snt.aligned"
# aligned_errors_style2 = "data/shakespeare/data/align/plays/merged/errors_original.snt.aligned"

# aligned_hamlet_style1 = "data/shakespeare/data/align/plays/merged/hamlet_modern.snt.aligned"
# aligned_hamlet_style2 = "data/shakespeare/data/align/plays/merged/hamlet_original.snt.aligned"

# aligned_henryv_style1 = "data/shakespeare/data/align/plays/merged/henryv_modern.snt.aligned"
# aligned_henryv_style2 = "data/shakespeare/data/align/plays/merged/henryv_original.snt.aligned"

# aligned_juliuscaesar_style1 = "data/shakespeare/data/align/plays/merged/juliuscaesar_modern.snt.aligned"
# aligned_juliuscaesar_style2 = "data/shakespeare/data/align/plays/merged/juliuscaesar_original.snt.aligned"

# style1_tuples = unidecode.unidecode(open(aligned_style1).read())
# style2_tuples = unidecode.unidecode(open(aligned_style2).read())

# style1_tuples = style1_tuples.split("\n")
# style2_tuples = style2_tuples.split("\n")

In [62]:
# print("{}\n{}\n".format(agg_original_tuples[0], agg_modern_tuples[0]))
# print("{}\n{}\n".format(agg_original_tuples[5000], agg_modern_tuples[5000]))
# print("{}\n{}\n".format(agg_original_tuples[9000], agg_modern_tuples[9000]))
# print("{}\n{}\n".format(agg_original_tuples[18000], agg_modern_tuples[18000]))


In [63]:
# in-class = original = '1'
# raw -> (raw text, label)
import numpy as np

style1_np = np.array(agg_original_tuples)
style1_labels = np.array(['1' for x in range(len(agg_original_tuples))])
X = np.vstack((style1_np, style1_labels))

style2_np = np.array(agg_modern_tuples)
style2_labels = np.array(['0' for x in range(len(agg_modern_tuples))])
X2 = np.vstack((style2_np, style2_labels))

X = np.hstack((X,X2))
raw = np.transpose(X)

idx_col = [str(x) for x in range(len(raw))]
idx_col = np.array(idx_col).reshape(-1,1)
print(idx_col.shape)
print(raw.shape)
raw = np.hstack((idx_col, raw))
raw.shape

(6566, 1)
(6566, 2)


(6566, 3)

In [64]:
raw[2000]

array(['2000', 'I long with all my heart to see the prince.', '1'],
      dtype='<U497')

In [65]:
from sklearn.feature_extraction import DictVectorizer
import spacy
import scipy.sparse
nlp = spacy.load('en_core_web_sm')
print

vectorizer = DictVectorizer()
features = []

# feature engineering
    # construct index column
idx_col = np.array([raw[:,0]])
idx_col = idx_col.astype('f', copy=True)

#sent_len_feature = np.array([-1 for x in range(len(raw))])
raw_text_col = 1
LABEL_COL = -1

for row_idx,_ in enumerate(raw):
    feature_dict = {}
    pos_dict = {'NOUN_count': 0, 'ADV_count': 0, 'VERB_count': 0, 
                'ADJ_count': 0, 'adv_verb_ratio': 0, 'adj_noun_ratio': 0}
    txt = str(raw[row_idx][raw_text_col])
    
    feature_dict['sent_len'] = len(txt)
    doc = nlp(txt)
    for token in doc:
        pos = token.pos_
        pos_dict[pos+'_count'] = pos_dict.get(pos+'_count', 0) + 1

    if pos_dict['NOUN_count'] == 0 or pos_dict['ADJ_count'] == 0:
        pos_dict['adj_noun_ratio'] = 0
    else:
        pos_dict['adj_noun_ratio'] = pos_dict['NOUN_count'] / pos_dict['ADJ_count'] 

    if pos_dict['VERB_count'] == 0 or pos_dict['ADV_count'] == 0:
        pos_dict['adv_verb_ratio'] = 0
    else:
        pos_dict['adv_verb_ratio'] = pos_dict['VERB_count'] / pos_dict['ADV_count'] 
        
    feature_dict.update(pos_dict)
    
    if row_idx % 1000 == 0:
        print("{} gold: {} {}\n".format(txt, raw[row_idx][LABEL_COL], feature_dict) )
        
    features.append(feature_dict)

labels_col = np.array([raw[:,LABEL_COL].astype(float)])

X = vectorizer.fit_transform(features)
print(type(X))
print(X.shape)
print(idx_col.T.shape)
X = scipy.sparse.hstack(( idx_col.T, X, labels_col.T ))
X = X.todense()
# X = np.transpose(X)
# X = X.astype('f', copy=True)
# print(X[0:5])
# print(X[3700:3705])

What's the matter, lieutenant? gold: 1 {'sent_len': 30, 'NOUN_count': 3, 'ADV_count': 0, 'VERB_count': 1, 'ADJ_count': 0, 'adv_verb_ratio': 0, 'adj_noun_ratio': 0, 'DET_count': 1, 'PUNCT_count': 2}

Therefore, good Emilia, Give me my nightly wearing, and adieu. gold: 1 {'sent_len': 62, 'NOUN_count': 1, 'ADV_count': 2, 'VERB_count': 2, 'ADJ_count': 2, 'adv_verb_ratio': 1.0, 'adj_noun_ratio': 0.5, 'PUNCT_count': 4, 'PROPN_count': 1, 'PRON_count': 1, 'CCONJ_count': 1}

I long with all my heart to see the prince. gold: 1 {'sent_len': 43, 'NOUN_count': 2, 'ADV_count': 1, 'VERB_count': 1, 'ADJ_count': 2, 'adv_verb_ratio': 1.0, 'adj_noun_ratio': 1.0, 'PRON_count': 1, 'ADP_count': 1, 'PART_count': 1, 'DET_count': 1, 'PUNCT_count': 1}

On me, that halts and am misshapen thus? gold: 1 {'sent_len': 40, 'NOUN_count': 1, 'ADV_count': 1, 'VERB_count': 2, 'ADJ_count': 0, 'adv_verb_ratio': 2.0, 'adj_noun_ratio': 0, 'ADP_count': 2, 'PRON_count': 1, 'PUNCT_count': 2, 'CCONJ_count': 1}

Yes, that's for s

In [66]:
X[1:2]

matrix([[ 1.,  0.,  1.,  0.,  0.,  2.,  0.,  3.,  0.,  0.,  1.,  0.,  1.,
          0.,  2.,  0.,  0.,  0., 42.,  1.]])

In [67]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
import time
import math
###############
# training ####
###############
# test breakout: 3500, 210
train_val_split_delim = math.floor(len(X) * .8)
val_test_split_delim = math.floor(len(X) * .9)
print("total rows: {} train val split delim: {} val test split delim: {}".format(len(X), train_val_split_delim, val_test_split_delim))
# numpy note: addressing [:delim] is exclusive of delim, but
#                         [delim:] is inclusive of delim
last_feat_col = X.shape[1] -2
X = np.random.permutation(X)

X_train = X[0:train_val_split_delim,1:last_feat_col ]
Y_train = X[0:train_val_split_delim,LABEL_COL]

X_val = X[train_val_split_delim:val_test_split_delim,1:last_feat_col ]
Y_val = X[train_val_split_delim:val_test_split_delim,LABEL_COL]

X_test = X[val_test_split_delim:,1:last_feat_col ]
Y_test = X[val_test_split_delim:,LABEL_COL]


#mdl = LogisticRegression(random_state=123)
mdl = MultinomialNB()
mdl.fit(X_train, Y_train)

np.set_printoptions(suppress=True)


total rows: 6566 train val split delim: 5252 val test split delim: 5909


In [68]:
mdl.classes_

array([0., 1.])

In [78]:
# evaluation ###
################
#second class is original

val_indices = X[train_val_split_delim:val_test_split_delim,0]

def weighted_eval(raw, preds, val_indices, loss='SSE'):
    if len(preds) != len(val_indices):
        raise ArgumentError("weighted_eval: list length mismatch!")
        
    results = []
    
    for (idx, pred) in zip(val_indices, preds):
        gold_label = int(raw[int(idx)][-1])
        #pred_label = int(pred)
        #print("int'd idx: {} gold label: {} pred label: {}".format(int(idx), gold_label, pred_label))

        predicted_class = 0 if (pred[0] > pred[1]) else 1
        
        if gold_label == pred_label:
              results.append( (1 - pred[predicted_class])**2 )
        else:
              results.append( (pred[predicted_class])**2 )
    print(results[:20])
    return sum(results)/len(preds)
    
#binary evaluation
# preds = mdl.predict(X_test)
# 
# test_indices = X[val_test_split_delim:,0]
# if len(preds) != len(val_indices):
#     raise ValueError("unmatched arrays")
    
# results = []
# for (idx, pred) in zip(val_indices,preds):
#     gold_label = int(raw[int(idx)][-1])
#     pred_label = int(pred)
#     #print("int'd idx: {} gold label: {} pred label: {}".format(int(idx), gold_label, pred_label))
    
#     if gold_label == pred_label:
#           results.append(1)
#     else:
#         results.append(0)
# results[:5]
preds = mdl.predict_proba(X_test)
brier = weighted_eval(raw, preds, val_indices, loss='brier')
print("brier score:", brier)

[0.08875201783296359, 0.06207592467547917, 0.18068949024656478, 0.03350168830167186, 0.10201584225445073, 0.2812511258025143, 0.4053102290056527, 0.11757038636081334, 0.026746773362279472, 0.1947764703124586, 0.25583708594783405, 0.4185305139215705, 0.04694867223809446, 0.241283257880316, 0.118505685927454, 0.21359455454502094, 0.05506834808928139, 0.18571651931728084, 0.0012267252154180722, 0.40485019448912923]
brier score: {} 0.29244853370098184


In [ ]:
# test_len = len(preds)
# accuracy = sum(preds)/float(test_len)
# print("accuracy: {}".format(accuracy))

#othello accuracy: 0.7428571428571429. but maybe not permuted
#all: accuracy: 0.328237942884228

#pos, no bleu, 2 plays
# Mult NB: .519